# Hybrid Model - Metrics

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import os 
import skimage.measure as meas
import nibabel as nib
import natsort
import glob

/home/roberto/Documents/virtualenv/venv04/local/lib/python2.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
/home/roberto/Documents/virtualenv/venv04/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
orig = "/media/roberto/DATA/GDrive/MR-Data/Normative-Raw-Data/Test/*.npy"
hybrid = "../Hybrid-rec/*.npy"

orig = glob.glob(orig)
hybrid = glob.glob(hybrid)

orig = natsort.natsorted(orig)
hybrid = natsort.natsorted(hybrid)
print orig[0]
print hybrid[0]
ref = np.zeros((170*len(orig),256,256))
hyb = np.zeros((170*len(hybrid),256,256))

/media/roberto/DATA/GDrive/MR-Data/Normative-Raw-Data/Test/e14296s4_P09216.7.npy
../Hybrid-rec/e14296s4_P09216_hybrid.npy


In [3]:
for ii in xrange(len(hybrid)):
    aux1 = np.load(orig[ii])
    aux1 = np.abs(np.fft.ifft2(aux1[:,:,:,0]+1j*aux1[:,:,:,1])).astype(np.float64)
    ref[ii*170:(ii+1)*170,:,:] = aux1
    aux2 = np.load(hybrid[ii]).astype(np.float64)
    hyb[ii*170:(ii+1)*170,:,:] = aux2

In [4]:
metrics = np.zeros((hyb.shape[0],3))
for ii in xrange(hyb.shape[0]):  
    metrics[ii,0] = meas.compare_ssim(ref[ii].ravel(),hyb[ii].ravel(),win_size = ref[ii].size-1)
    metrics[ii,1] = meas.compare_nrmse(ref[ii],hyb[ii],'min-max')
    metrics[ii,2] = meas.compare_psnr(ref[ii],hyb[ii],data_range=(ref[ii].max()-ref[ii].min())) 

In [5]:
metrics[:,1] = metrics[:,1]*100
print "Hybrid"
print "%.3f +/- %.3f" %(metrics[:,0].mean(),metrics[:,0].std())
print "%.3f +/- %.3f" %(metrics[:,1].mean(),metrics[:,1].std())
print "%.3f +/- %.3f" %(metrics[:,2].mean(),metrics[:,2].std())

Hybrid
0.978 +/- 0.077
1.888 +/- 1.104
35.229 +/- 3.206
